In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Problem Statement

**Data Analysis case study**

I am working on a logistic problem where they have faced cancellation of orders in their deliveries. 
Their main goal is to be the 1st prefered delivery partner in Africa.

**Challenge 2**

-There's been deliberate deletion of data
1. Use the data 2, missed data to predict cancellation reasons for all orders with missing data.
2. Document steps and method used.

How to predict cancellation reasons
Our algorithm will be a multi-class classification machine learning algorithm which involves the following steps:

1. Importing libraries
2. Fetching the dataset
3. Creating the dependent variable class
4. Extracting features and output
5. Train-Test dataset splitting (include test dataset as holdout data)
6. Feature scaling
7. Training the model
8. Calculating the model score using the metric deemed fit based on the problem
9. Saving the model for predicting cancellation reasons on the test dataset

In [51]:
##for faster data processing
import numba as nb
import dask as da

#for data manipulation 
import pandas as pd
import numpy as np

##for data visualizations, distribution  and prediction of the variables
from pandas_summary import DataFrameSummary
import scipy as sc
import sklearn as skl
import seaborn as sns

import random

# Import Dataset

In [111]:
query_df = pd.read_csv('../Sendy_data/prediction_df.csv', sep=',')
test_df = pd.read_csv('../Sendy_data/test_df.csv', sep=',')


In [112]:
query_df.head()

,index,personal_or_business,business_number,vehicle_type,standard_or_express,platform,cancel_reason,cancelled_by,placement_day,placement_time,...,distance_(km),pickup_lat,pickup_long,scheduled,order_timedelta,processing_timedelta,confirmation_timedelta,order_period,processing_period,confirmation_period
0,0,Personal,-1.0,Bike,Express,android,No driver has been allocated to my request.,Customer,1,2019-07-29 17:55:38,...,9,-1.313053,36.841437,OnDemAND,NaN,-1 days +23:59:03.000000000,NaN,0,0,0
1,1,Personal,-1.0,Bike,Express,android,My order is not ready.,Customer,1,2019-07-29 12:26:27,...,14,-1.318959,36.853394,OnDemAND,-1 days +23:58:51.000000000,-1 days +23:57:43.000000000,-1 days +23:58:52.000000000,0,0,0
2,2,Business,20680.0,Bike,Express,android,I placed the wrong locations.,Customer,1,2019-07-29 13:21:18,...,16,-1.313854,36.706867,OnDemAND,-1 days +23:59:44.000000000,-1 days +23:33:49.000000000,-1 days +23:34:05.000000000,0,0,0
3,3,Personal,-1.0,Bike,Express,android,My order is not ready.,Customer,1,2019-07-29 10:29:19,...,20,-1.206197,36.778776,OnDemAND,NaN,-1 days +23:59:50.000000000,NaN,0,0,0
4,4,Personal,-1.0,Bike,Standard,ios,No Partner,Staff,1,2019-07-29 14:34:05,...,11,-1.296958,36.808166,OnDemAND,-1 days +23:34:57.000000000,-1 days +22:44:24.000000000,-1 days +23:09:27.000000000,0,0,0


In [113]:
test_df.head()

,personal_or_business,business_number,vehicle_type,standard_or_express,platform,cancel_reason,cancelled_by,placement_day,placement_time,confirmation_day,confirmation_time,cancellation_day,cancellation_time,distance_(km),pickup_lat,pickup_long,scheduled
0,Personal,-1.0,Bike,Express,ios,NaN,Customer,1,15:23:44,1,16:40:47,1,16:43:50,4,-1.299370,36.822107,OnDemAND
1,Business,20680.0,Bike,Express,desktop,NaN,Staff,1,14:21:46,1,14:22:20,1,14:30:48,17,-1.264490,36.789303,OnDemAND
2,Business,17968.0,Bike,Express,desktop,NaN,Customer,1,15:13:20,1,NaN,1,15:56:34,18,-1.427056,36.961499,OnDemAND
3,Personal,-1.0,Bike,Standard,android,NaN,Customer,1,13:54:19,1,NaN,1,13:56:41,8,-1.301737,36.743936,OnDemAND
4,Personal,-1.0,Bike,Standard,android,NaN,Staff,1,13:24:05,1,15:57:20,1,17:14:46,13,-1.308098,36.920464,Scheduled


#### Deal with outliers

In [114]:
##z-test (check outliers using the z-test)
from scipy import stats
import numpy as np
z = np.abs(stats.zscore(query_df['distance_(km)']))

print (z)

threshold = 3
print(np.where(z > 3))

[0.10883219 0.00152599 0.04566926 ... 0.26638563 0.08676056 0.02054565]
(array([  42,  344,  356, 2816, 2954, 2955, 4203, 4208, 4250, 4259, 4266,
       4301, 4304, 4338, 5490, 5495, 5502, 5503, 5519, 5692, 5775, 6107,
       6134]),)


In [115]:
query_df.shape

(6393, 24)

## Assign numbers for string features

In [116]:
# traversing through personal or business orders 
# column of dataFrame and writing values where  condition matches. 
query_df['personal_or_business'][query_df['personal_or_business'] == 'Personal'] = 1
query_df['personal_or_business'][query_df['personal_or_business'] == 'Business'] = 2


/home/dancilla/PYTHON_37/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/dancilla/PYTHON_37/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [117]:
# traversing through types of vehicles 
# column of dataFrame and writing values where  condition matches. 
query_df['vehicle_type'][query_df['vehicle_type'] == 'Bike'] = 1
query_df['vehicle_type'][query_df['vehicle_type'] == 'Standard Bike'] = 2


/home/dancilla/PYTHON_37/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/dancilla/PYTHON_37/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [118]:
# traversing through types of order 
# column of dataFrame and writing values where  condition matches. 
query_df['standard_or_express'][query_df['standard_or_express'] == 'Express'] = 1
query_df['standard_or_express'][query_df['standard_or_express'] == 'Standard'] = 2


/home/dancilla/PYTHON_37/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/dancilla/PYTHON_37/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [119]:
# traversing through who cancelled the orders
# column of dataFrame and writing values where  condition matches. 
query_df['cancelled_by'][query_df['cancelled_by'] == 'Customer'] = 1
query_df['cancelled_by'][query_df['cancelled_by'] == 'Staff'] = 2


/home/dancilla/PYTHON_37/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/dancilla/PYTHON_37/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [120]:
# traversing through types of indication of whether order was on demand or scheduled
# column of dataFrame and writing values where  condition matches. 
query_df['scheduled'][query_df['scheduled'] == 'OnDemAND'] = 1
query_df['scheduled'][query_df['scheduled'] == 'Scheduled'] = 2


/home/dancilla/PYTHON_37/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/dancilla/PYTHON_37/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [121]:
# traversing through types of platforms used 
# column of dataFrame and writing values where  condition matches. 
query_df['platform'][query_df['platform'] == 'android'] = 1
query_df['platform'][query_df['platform'] == 'ios'] = 2
query_df['platform'][query_df['platform'] == 'desktop'] = 3
query_df['platform'][query_df['platform'] == 'API'] = 4
query_df['platform'][query_df['platform'] == 'peer_web'] = 5


/home/dancilla/PYTHON_37/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/dancilla/PYTHON_37/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/dancilla/PYTHON_37/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide

In [122]:
# traversing through types of reasons of cancelling orders
# column of dataFrame and writing values where  condition matches. 
query_df['cancel_reason'][query_df['cancel_reason'] == 'No driver has been allocated to my request.'] = 1
query_df['cancel_reason'][query_df['cancel_reason'] == 'My order is not ready.'] = 2
query_df['cancel_reason'][query_df['cancel_reason'] == 'I placed the wrong locations.'] = 3
query_df['cancel_reason'][query_df['cancel_reason'] == 'No Partner'] = 4
query_df['cancel_reason'][query_df['cancel_reason'] == 'Customer error'] = 5
query_df['cancel_reason'][query_df['cancel_reason'] == 'The driver is too far away.'] = 6
query_df['cancel_reason'][query_df['cancel_reason'] == 'Delay'] = 7



/home/dancilla/PYTHON_37/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/dancilla/PYTHON_37/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/dancilla/PYTHON_37/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide

### Change timelines to pandas datetime

In [123]:
query_df['placement_time'] = pd.to_datetime(query_df['placement_time'])
query_df['confirmation_time'] = pd.to_datetime(query_df['confirmation_time'])
query_df['cancellation_time'] = pd.to_datetime(query_df['cancellation_time'])


In [124]:
query_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6393 entries, 0 to 6392
Data columns (total 24 columns):
index                     6393 non-null int64
personal_or_business      6393 non-null object
business_number           6393 non-null float64
vehicle_type              6393 non-null object
standard_or_express       6393 non-null object
platform                  6393 non-null object
cancel_reason             6393 non-null object
cancelled_by              6393 non-null object
placement_day             6393 non-null int64
placement_time            6393 non-null datetime64[ns]
confirmation_day          6393 non-null int64
confirmation_time         2879 non-null datetime64[ns]
cancellation_day          6393 non-null int64
cancellation_time         6393 non-null datetime64[ns]
distance_(km)             6393 non-null int64
pickup_lat                6393 non-null float64
pickup_long               6393 non-null float64
scheduled                 6393 non-null object
order_timedelta          

## Split data

In [125]:
query_df.keys()

Index(['index', 'personal_or_business', 'business_number', 'vehicle_type',
       'standard_or_express', 'platform', 'cancel_reason', 'cancelled_by',
       'placement_day', 'placement_time', 'confirmation_day',
       'confirmation_time', 'cancellation_day', 'cancellation_time',
       'distance_(km)', 'pickup_lat', 'pickup_long', 'scheduled',
       'order_timedelta', 'processing_timedelta', 'confirmation_timedelta',
       'order_period', 'processing_period', 'confirmation_period'],
      dtype='object')

In [126]:
test_df.keys()

Index(['personal_or_business', 'business_number', 'vehicle_type',
       'standard_or_express', 'platform', 'cancel_reason', 'cancelled_by',
       'placement_day', 'placement_time', 'confirmation_day',
       'confirmation_time', 'cancellation_day', 'cancellation_time',
       'distance_(km)', 'pickup_lat', 'pickup_long', 'scheduled'],
      dtype='object')

In [129]:
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.externals import joblib


#Choose the features which will be used for building the model
columns = ['personal_or_business', 'business_number', 'vehicle_type',
       'standard_or_express', 'platform', 'cancelled_by',
       'placement_day', 'confirmation_day', 'cancellation_day',
       'distance_(km)', 'pickup_lat', 'pickup_long', 'scheduled']

holdout = test_df # from now on we will refer to this
               # dataframe as the holdout data
X = query_df[columns]
y = query_df['cancel_reason']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,random_state=0)

**In this case, since we are predicting multiple values, we need to use the multiclass classification using
using Random Forest on Scikit-Learn Library**


## Feature Scaling

In [130]:
# Feature Scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

/home/dancilla/PYTHON_37/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/dancilla/PYTHON_37/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/dancilla/PYTHON_37/lib/python3.6/site-packages/ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64, object were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.


## Training the model

In [131]:
# Fitting Random Forest Classification to the Training set
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 42)
classifier.fit(X_train, y_train)

ValueError: Unknown label type: 'unknown'

In [15]:
#splitting into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(query_df, query_df, random_state = 0)



NameError: name 'df' is not defined